# E06 Architecture Experiments - Win Rate Analysis

**Objective**: Analyze win rates from 24 CNN architecture variants running in E06 experiments

This notebook parses log files to extract and visualize training progress.

In [56]:
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.ndimage import uniform_filter1d

## Parse Log Files

Extract win rates from contest results in log files.

In [57]:
# Get all E06 log files
log_dir = Path("../../exe")
log_files = sorted(log_dir.glob("E06_ARCH*.log"))

print(f"Found {len(log_files)} log files")
for log in log_files[:5]:
    print(f"  {log.name}")

Found 24 log files
  E06_ARCHV01_BASELINE.log
  E06_ARCHV02_WIDER_FC.log
  E06_ARCHV03_MUCH_WIDER_FC.log
  E06_ARCHV04_DEEPER_CONV.log
  E06_ARCHV05_DEEP_WIDE.log


In [58]:
def parse_log_file(log_path):
    """Extract epoch and win rate data from log file."""
    pattern_epoch = re.compile(r"Contest results after epoch (\d+)")
    # Fixed pattern: the log format is 'draws': X, 'losses': Y, 'wins': Z (not 'total')
    pattern_wr_good = re.compile(
        r"'bot_good_WR_B': {'draws': (\d+), 'losses': (\d+), 'wins': (\d+)}"
    )
    pattern_wr_random = re.compile(
        r"'bot_random': {'draws': (\d+), 'losses': (\d+), 'wins': (\d+)}"
    )

    epochs = []
    win_rates_good = []
    win_rates_random = []

    try:
        with open(log_path, "r") as f:
            content = f.read()

        lines = content.split("\n")
        current_epoch = None
        temp_wr_good = None
        temp_wr_random = None

        for line in lines:
            epoch_match = pattern_epoch.search(line)
            if epoch_match:
                current_epoch = int(epoch_match.group(1))

            wr_match_good = pattern_wr_good.search(line)
            if wr_match_good:
                draws = int(wr_match_good.group(1))
                losses = int(wr_match_good.group(2))
                wins = int(wr_match_good.group(3))
                total = draws + losses + wins
                temp_wr_good = wins / total if total > 0 else 0

            wr_match_random = pattern_wr_random.search(line)
            if wr_match_random:
                draws = int(wr_match_random.group(1))
                losses = int(wr_match_random.group(2))
                wins = int(wr_match_random.group(3))
                total = draws + losses + wins
                temp_wr_random = wins / total if total > 0 else 0

            # When we have both good and random win rates for an epoch, save them
            if (
                current_epoch is not None
                and temp_wr_good is not None
                and temp_wr_random is not None
            ):
                epochs.append(current_epoch)
                win_rates_good.append(temp_wr_good)
                win_rates_random.append(temp_wr_random)
                current_epoch = None
                temp_wr_good = None
                temp_wr_random = None

    except Exception as e:
        print(f"Error reading {log_path.name}: {e}")

    return epochs, win_rates_good, win_rates_random


# Parse all log files
variant_data = {}
for log_file in log_files:
    variant = log_file.stem.replace("E06_ARCH", "")
    epochs, win_rates_good, win_rates_random = parse_log_file(log_file)
    if epochs:
        variant_data[variant] = {
            "epochs": epochs,
            "win_rates_good": win_rates_good,
            "win_rates_random": win_rates_random,
        }
        print(
            f"{variant}: {len(epochs)} epochs, latest WR_good={win_rates_good[-1]:.3f}, WR_random={win_rates_random[-1]:.3f}"
        )

print(f"\nSuccessfully parsed {len(variant_data)} variants")

V01_BASELINE: 410 epochs, latest WR_good=0.367, WR_random=0.533
V02_WIDER_FC: 406 epochs, latest WR_good=0.300, WR_random=0.467
V03_MUCH_WIDER_FC: 392 epochs, latest WR_good=0.433, WR_random=0.400
V04_DEEPER_CONV: 378 epochs, latest WR_good=0.367, WR_random=0.567
V05_DEEP_WIDE: 367 epochs, latest WR_good=0.300, WR_random=0.533
V06_RICH_PIECE: 410 epochs, latest WR_good=0.433, WR_random=0.600
V06_RICH_PIECE: 410 epochs, latest WR_good=0.433, WR_random=0.600
V07_LOW_DROPOUT: 411 epochs, latest WR_good=0.500, WR_random=0.600
V08_NO_DROPOUT: 413 epochs, latest WR_good=0.333, WR_random=0.367
V09_MEGA: 349 epochs, latest WR_good=0.467, WR_random=0.533
V10_NARROW_DEEP: 393 epochs, latest WR_good=0.300, WR_random=0.567
V11_WIDE_SHALLOW: 396 epochs, latest WR_good=0.500, WR_random=0.600
V12_BALANCED: 374 epochs, latest WR_good=0.500, WR_random=0.467
V13_VERY_DEEP: 325 epochs, latest WR_good=0.500, WR_random=0.667
V07_LOW_DROPOUT: 411 epochs, latest WR_good=0.500, WR_random=0.600
V08_NO_DROPOUT:

## Visualize Win Rate Progress

In [59]:
# Categorize variants into 4 segments based on architecture type
segments = {
    "Baseline & Bottleneck": [
        "V01_BASELINE",
        "V02_WIDER_FC",
        "V03_MUCH_WIDER_FC",
        "V15_ULTRA_WIDE_FC",
        "V22_MINIMAL",
    ],
    "Depth Variations": [
        "V04_DEEPER_CONV",
        "V05_DEEP_WIDE",
        "V10_NARROW_DEEP",
        "V11_WIDE_SHALLOW",
        "V13_VERY_DEEP",
        "V17_DEEP_NARROW",
        "V18_WIDE_DEEP",
        "V23_PROGRESSIVE",
    ],
    "Piece Embedding": ["V06_RICH_PIECE", "V14_HUGE_PIECE", "V19_PIECE_RICH_DEEP"],
    "Dropout & Balanced": [
        "V07_LOW_DROPOUT",
        "V08_NO_DROPOUT",
        "V09_MEGA",
        "V12_BALANCED",
        "V16_TINY_DROPOUT",
        "V20_SQUEEZE",
        "V21_BALANCED_HEAVY",
        "V24_ULTRA_BALANCED",
    ],
}

# Assign variants to segments
variant_segments = {}
for segment_name, variants in segments.items():
    for v in variants:
        variant_segments[v] = segment_name

print("Segment assignment:")
for segment, variants in segments.items():
    print(f"\n{segment}: {len(variants)} variants")
    print(f"  {', '.join(variants)}")

Segment assignment:

Baseline & Bottleneck: 5 variants
  V01_BASELINE, V02_WIDER_FC, V03_MUCH_WIDER_FC, V15_ULTRA_WIDE_FC, V22_MINIMAL

Depth Variations: 8 variants
  V04_DEEPER_CONV, V05_DEEP_WIDE, V10_NARROW_DEEP, V11_WIDE_SHALLOW, V13_VERY_DEEP, V17_DEEP_NARROW, V18_WIDE_DEEP, V23_PROGRESSIVE

Piece Embedding: 3 variants
  V06_RICH_PIECE, V14_HUGE_PIECE, V19_PIECE_RICH_DEEP

Dropout & Balanced: 8 variants
  V07_LOW_DROPOUT, V08_NO_DROPOUT, V09_MEGA, V12_BALANCED, V16_TINY_DROPOUT, V20_SQUEEZE, V21_BALANCED_HEAVY, V24_ULTRA_BALANCED


In [60]:
# Create 4 subplots using Plotly
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=list(segments.keys()),
    vertical_spacing=0.12,
    horizontal_spacing=0.08,
)

colors = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
]

for idx, (segment_name, variant_list) in enumerate(segments.items()):
    row = idx // 2 + 1
    col = idx % 2 + 1

    color_idx = 0
    for variant in variant_list:
        if variant in variant_data:
            data = variant_data[variant]
            epochs = np.array(data["epochs"])
            win_rates_good = np.array(data["win_rates_good"])
            win_rates_random = np.array(data["win_rates_random"])

            color = colors[color_idx % len(colors)]

            # Smooth the signals for overlay
            if len(win_rates_good) >= 10:
                smoothed_good = uniform_filter1d(
                    win_rates_good, size=10, mode="nearest"
                )
                smoothed_random = uniform_filter1d(
                    win_rates_random, size=10, mode="nearest"
                )
            else:
                smoothed_good = win_rates_good
                smoothed_random = win_rates_random

            # Add SMOOTHED win rate vs bot_good_WR_B (line)
            fig.add_trace(
                go.Scatter(
                    x=epochs,
                    y=smoothed_good,
                    name=f"{variant}",
                    mode="lines",
                    line=dict(color=color, width=2),
                    opacity=0.8,
                    legendgroup=variant,
                    showlegend=True,
                    hovertemplate=f"{variant}<br>Epoch: %{{x}}<br>WR vs good (smoothed): %{{y:.3f}}<extra></extra>",
                ),
                row=row,
                col=col,
            )

            # Add SMOOTHED win rate vs bot_random (very transparent, dotted)
            fig.add_trace(
                go.Scatter(
                    x=epochs,
                    y=smoothed_random,
                    name=f"{variant} (vs random)",
                    mode="lines",
                    line=dict(color=color, width=1.5, dash="dot"),
                    opacity=0.2,
                    legendgroup=variant,
                    showlegend=False,
                    hovertemplate=f"{variant}<br>Epoch: %{{x}}<br>WR vs random (smoothed): %{{y:.3f}}<extra></extra>",
                ),
                row=row,
                col=col,
            )

            color_idx += 1

# Update axes
for i in range(1, 5):
    row = (i - 1) // 2 + 1
    col = (i - 1) % 2 + 1
    fig.update_xaxes(title_text="Epoch", row=row, col=col, gridcolor="lightgray")
    fig.update_yaxes(
        title_text="Win Rate", row=row, col=col, range=[0, 1], gridcolor="lightgray"
    )

# Update layout
fig.update_layout(
    title_text="E06 Architecture Experiments - Win Rate Progress by Category (Live)<br><sub>Smoothed (solid=vs bot_good_WR_B, dotted=vs bot_random)</sub>",
    height=900,
    width=1400,
    showlegend=True,
    legend=dict(
        orientation="v", yanchor="top", y=1, xanchor="left", x=1.01, font=dict(size=9)
    ),
    hovermode="closest",
)

fig.write_html("E06_win_rate_by_segment_interactive.html")
fig.show()

print(
    f"\nInteractive plot saved to analysis/E06_architecture/E06_win_rate_by_segment_interactive.html"
)


Interactive plot saved to analysis/E06_architecture/E06_win_rate_by_segment_interactive.html


## Summary Statistics

In [61]:
# Create summary DataFrame
summary = []
for variant, data in variant_data.items():
    wr_good = data["win_rates_good"]
    wr_random = data["win_rates_random"]
    summary.append(
        {
            "Variant": variant,
            "Epochs": len(wr_good),
            "Latest WR (good)": wr_good[-1] if wr_good else 0,
            "Latest WR (random)": wr_random[-1] if wr_random else 0,
            "Max WR (good)": max(wr_good) if wr_good else 0,
            "Mean WR (good)": np.mean(wr_good) if wr_good else 0,
            "Std WR (good)": np.std(wr_good) if wr_good else 0,
        }
    )

df_summary = pd.DataFrame(summary).sort_values("Latest WR (good)", ascending=False)
print("E06 Variant Summary (sorted by latest win rate vs bot_good_WR_B):")
print("=" * 100)
print(df_summary.to_string(index=False))
print("=" * 100)

E06 Variant Summary (sorted by latest win rate vs bot_good_WR_B):
            Variant  Epochs  Latest WR (good)  Latest WR (random)  Max WR (good)  Mean WR (good)  Std WR (good)
 V24_ULTRA_BALANCED     367          0.533333            0.466667       0.633333        0.372934       0.085596
        V22_MINIMAL     415          0.533333            0.466667       0.600000        0.369960       0.084596
    V07_LOW_DROPOUT     411          0.500000            0.600000       0.733333        0.381265       0.090822
       V12_BALANCED     374          0.500000            0.466667       0.633333        0.373351       0.092819
   V11_WIDE_SHALLOW     396          0.500000            0.600000       0.600000        0.370455       0.081303
      V13_VERY_DEEP     325          0.500000            0.666667       0.600000        0.363077       0.088312
           V09_MEGA     349          0.466667            0.533333       0.633333        0.368959       0.088540
     V06_RICH_PIECE     410          0

## Weight Dynamics Analysis

Analyze weight changes across training for selected variants to understand learning dynamics.

In [62]:
import torch
from collections import defaultdict

# Select top 3 variants by latest win rate for weight analysis
top_variants = df_summary.head(3)["Variant"].tolist()
print(f"Analyzing weight dynamics for top 3 variants: {top_variants}")

# Get checkpoint directories
checkpoint_base = Path("../../CHECKPOINTS")
weight_data = defaultdict(lambda: defaultdict(list))

for variant in top_variants:
    # Find checkpoint folder for this variant
    ckpt_folders = list(checkpoint_base.glob(f"E06_ARCH{variant}*"))

    if not ckpt_folders:
        print(f"No checkpoints found for {variant}")
        continue

    ckpt_folder = ckpt_folders[0]
    print(f"\n{variant}: {ckpt_folder.name}")

    # Get all checkpoint files
    ckpt_files = sorted(ckpt_folder.glob("*.pt"))

    if len(ckpt_files) < 2:
        print(f"  Not enough checkpoints ({len(ckpt_files)})")
        continue

    # Analyze ALL checkpoints for full temporal resolution
    ckpt_files_sample = ckpt_files
    print(f"  Analyzing {len(ckpt_files_sample)} checkpoints (all)")

    prev_state = None
    for ckpt_file in ckpt_files_sample:
        # Extract epoch from filename (format: E06_ARCH{variant}_E_{epoch:04d}.pt)
        epoch_str = ckpt_file.stem.split("_E_")[-1]
        epoch = int(epoch_str)

        # Load checkpoint
        try:
            state_dict = torch.load(ckpt_file, map_location="cpu")

            if prev_state is not None:
                # Calculate weight changes for each layer
                for layer_name, curr_weights in state_dict.items():
                    if layer_name in prev_state:
                        prev_weights = prev_state[layer_name]

                        # Calculate Frobenius norm of difference
                        delta = curr_weights - prev_weights
                        delta_norm = torch.norm(delta).item()

                        # Calculate relative change
                        weight_norm = torch.norm(curr_weights).item()
                        relative_change = (
                            delta_norm / weight_norm if weight_norm > 0 else 0
                        )

                        weight_data[variant][layer_name].append(
                            {
                                "epoch": epoch,
                                "delta_norm": delta_norm,
                                "relative_change": relative_change,
                                "weight_norm": weight_norm,
                            }
                        )

            prev_state = state_dict

        except Exception as e:
            print(f"  Error loading {ckpt_file.name}: {e}")
            continue

print(f"\nWeight analysis complete for {len(weight_data)} variants")

Analyzing weight dynamics for top 3 variants: ['V24_ULTRA_BALANCED', 'V22_MINIMAL', 'V07_LOW_DROPOUT']

V24_ULTRA_BALANCED: E06_ARCHV24_ULTRA_BALANCED
  Analyzing 369 checkpoints (all)


C:\Users\ALLAN TURING\AppData\Local\Temp\ipykernel_38836\2520834166.py:42: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.




V22_MINIMAL: E06_ARCHV22_MINIMAL
  Analyzing 417 checkpoints (all)

V07_LOW_DROPOUT: E06_ARCHV07_LOW_DROPOUT
  Analyzing 413 checkpoints (all)

Weight analysis complete for 3 variants

Weight analysis complete for 3 variants


In [63]:
# Plot weight dynamics for each variant
fig = make_subplots(
    rows=len(weight_data),
    cols=1,
    subplot_titles=[
        f"{variant} - Layer Weight Changes" for variant in weight_data.keys()
    ],
    vertical_spacing=0.15,
)

for idx, (variant, layer_data) in enumerate(weight_data.items(), 1):
    # Get unique layer names
    layer_names = list(layer_data.keys())

    for layer_name in layer_names:
        changes = layer_data[layer_name]
        epochs = [c["epoch"] for c in changes]
        delta_norms = [c["delta_norm"] for c in changes]

        fig.add_trace(
            go.Scatter(
                x=epochs,
                y=delta_norms,
                name=layer_name,
                mode="lines+markers",
                marker=dict(size=4),
                hovertemplate=f"{layer_name}<br>Epoch: %{{x}}<br>Δnorm: %{{y:.4f}}<extra></extra>",
            ),
            row=idx,
            col=1,
        )

    fig.update_xaxes(title_text="Epoch", row=idx, col=1, gridcolor="lightgray")
    fig.update_yaxes(
        title_text="Weight Change (Δnorm)",
        row=idx,
        col=1,
        gridcolor="lightgray",
        type="log",
    )

fig.update_layout(
    title_text="E06 Weight Dynamics - Top 3 Variants",
    height=400 * len(weight_data),
    width=1200,
    showlegend=True,
    hovermode="closest",
)

fig.write_html("E06_weight_dynamics.html")
fig.show()

print(
    f"\nWeight dynamics plot saved to analysis/E06_architecture/E06_weight_dynamics.html"
)


Weight dynamics plot saved to analysis/E06_architecture/E06_weight_dynamics.html


In [64]:
# Summary statistics of weight changes
print("Weight Change Statistics by Layer and Variant")
print("=" * 100)

for variant, layer_data in weight_data.items():
    print(f"\n{variant}:")
    print("-" * 100)

    for layer_name, changes in layer_data.items():
        if not changes:
            continue

        delta_norms = [c["delta_norm"] for c in changes]
        relative_changes = [c["relative_change"] for c in changes]

        print(f"\n  {layer_name}:")
        print(
            f"    Δnorm - Mean: {np.mean(delta_norms):.6f}, Max: {max(delta_norms):.6f}, Min: {min(delta_norms):.6f}"
        )
        print(
            f"    Relative - Mean: {np.mean(relative_changes):.6f}, Max: {max(relative_changes):.6f}"
        )

        # Find epoch with largest change
        max_idx = delta_norms.index(max(delta_norms))
        max_epoch = changes[max_idx]["epoch"]
        print(f"    Max change at epoch: {max_epoch}")

print("\n" + "=" * 100)

Weight Change Statistics by Layer and Variant

V24_ULTRA_BALANCED:
----------------------------------------------------------------------------------------------------

  fc_in_piece.weight:
    Δnorm - Mean: 0.000407, Max: 0.002680, Min: 0.000112
    Relative - Mean: 0.000125, Max: 0.000826
    Max change at epoch: 17

  fc_in_piece.bias:
    Δnorm - Mean: 0.000198, Max: 0.002479, Min: 0.000032
    Relative - Mean: 0.000261, Max: 0.003270
    Max change at epoch: 17

  conv1.weight:
    Δnorm - Mean: 0.001606, Max: 0.016299, Min: 0.000375
    Relative - Mean: 0.000625, Max: 0.006345
    Max change at epoch: 19

  conv1.bias:
    Δnorm - Mean: 0.000200, Max: 0.002820, Min: 0.000014
    Relative - Mean: 0.001007, Max: 0.014269
    Max change at epoch: 18

  conv2.weight:
    Δnorm - Mean: 0.002672, Max: 0.029388, Min: 0.000523
    Relative - Mean: 0.000814, Max: 0.008952
    Max change at epoch: 19

  conv2.bias:
    Δnorm - Mean: 0.000207, Max: 0.002649, Min: 0.000021
    Relative - Me